In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('max_info_columns', 500)

import numpy as np

In [53]:
## Load dataframes

#Load Main Dataframe
df = pd.read_csv('diabetic_data.csv'
                ,sep = ','
                ,na_values = '?')
"""
Create subset of dataframe keeping the first encounter provided for each 'patient_nbr' value. This keeps the rows with
the lowest index value and the lowest encounter ID value. This is build on the assumption that encounter IDs are assigned
sequentially as admissions occur

Also excluding encounters with a discharge type that suggests a readmission is planned, unlikely or impossible

Keeping only one admission per patient is important for ensuring the observations are independent 
"""
df = df[~df['discharge_disposition_id'].isin([11,12,13,14,19,20,21])].sort_index(ascending = True).drop_duplicates(subset = ['patient_nbr'], keep = 'first')

# Load lookup tables
admitTypeDf = pd.read_excel('DiabeticDataMap.xlsx', sheet_name = 'admitType')
dischargeDispDf = pd.read_excel('DiabeticDataMap.xlsx', sheet_name = 'dischargeDisp')
admitSourceDf = pd.read_excel('DiabeticDataMap.xlsx', sheet_name = 'admitSource')

# Merge admit source id description, drop id column
df = df.merge(admitSourceDf, how ='inner', on = 'admission_source_id')
df.rename(columns = {'description' : 'admission_source'}, inplace = True); del df['admission_source_id']

# Merge admit type id description, drop id column
df = df.merge(admitTypeDf, how ='inner', on = 'admission_type_id')
df.rename(columns = {'description' : 'admission_type'}, inplace = True); del df['admission_type_id']

# Merge discharge disposition id description, drop id column
df = df.merge(dischargeDispDf, how ='inner', on = 'discharge_disposition_id')
df.rename(columns = {'description' : 'discharge_disposition'}, inplace = True); del df['discharge_disposition_id']

# Delete weight column as it is mostly null
del df['weight']

# identify category columns do be encoded by get_dummies. note #excluding columns 'diag1','diag2','diag3','payer_code','medical_specialty'
categoryCols = ['race','gender','age','max_glu_serum', 'A1Cresult'
,'metformin','repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide','glipizide','glyburide'
,'tolbutamide','pioglitazone','rosiglitazone','acarbose','miglitol','troglitazone','tolazamide','examide'
,'citoglipton', 'insulin','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone'
,'metformin-rosiglitazone','metformin-pioglitazone','change','diabetesMed','admission_source'
,'admission_type','discharge_disposition']

/home/petersontylerd/.pyenv/versions/3.6.2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
df = pd.get_dummies(df, columns = categoryCols)

In [62]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69988 entries, 0 to 69987
Data columns (total 156 columns):
encounter_id                                                                                                                       69988 non-null int64
patient_nbr                                                                                                                        69988 non-null int64
time_in_hospital                                                                                                                   69988 non-null int64
payer_code                                                                                                                         39574 non-null object
medical_specialty                                                                                                                  36334 non-null object
num_lab_procedures                                                                                                              

In [ ]:
drug features = the feature indicates whether the drug was prescribed or there was a change in the 
dosage. Values: 
“up” if the dosage was increased during the encounter, 
“down” if the dosage was decreased, 
“steady” if the dosage did not change, 
“no” if the drug was not prescribed

In [ ]:
get dummies
designate categorical columns
get count of unique values in categorical columns
